# Speeding up your code!

There are a lot of ways that your code can be improved to run faster or more efficiently. Some of these methods are simple stylistic tweaks that you can implement instantly, whereas others will take a little more thought and may only work in specific circumstances.

This tutorial will cover the following topics:
* Looping through lists
* List comprehension
* Working with vectors
* CPU parallelization 
* GPU programming in Python.

This tutorial will not go into depth about parallelization methods or CUDA programming in general, but will instead act as a survey to make you aware of potential routes to speed your code up and direct you to those resources.


# Looping through lists

When working with large data structures it is sometimes easy to get in the habit of writing a loop for every step of a process. Consider the following example. Assume we have a list of particle positions that we want to do some analysis on.

In [1]:
import numpy as np
positions = np.random.randn(40000,3).tolist()

Let's look at the shape of our list of particle positions and the first 30 values in the list.

In [2]:
print(np.shape(positions),'\n')

positions[:30][:]

(40000, 3) 



[[-1.010997633001826, -0.1333018287079396, -0.987424763386151],
 [-0.11295827450162073, -2.7996884434750946, -0.5705246123453378],
 [1.29349018649756, 1.8768419061011234, 0.8422020635423959],
 [-0.8607264221445806, 0.042544516312457654, -0.5637799183415761],
 [-1.219566718781759, -2.1086758349047825, -1.636880471785036],
 [0.8492939338765828, 0.35990459837909894, 0.6517898799297105],
 [1.857685033916256, -1.5942093271484838, -0.34306033041430545],
 [-0.7305423177330297, 1.3854314073948577, 0.4593562043976882],
 [1.139232240087528, -0.6034225773868718, -0.8560325653782608],
 [-1.7498729539864217, -0.319059779512917, 0.4910961792876506],
 [0.9276385582046336, -0.6172036157567746, -0.7463392483149577],
 [0.20155806238014187, 1.8338034010637059, -1.5763336669812253],
 [2.829300465484318, 1.6443720295444846, -1.8547229075546792],
 [0.4655379310025332, -0.4738636201892907, 0.35222292238047365],
 [0.12284872992828724, 0.8247784487684606, -0.0596888252926106],
 [1.113508274052741, -0.367529870

<d></d>

Notice that we have a list of 40,000 particle positions written in a 3D coordinate system. If we look closely we will notice that this is a list of coordinate lists. That is, each particle has a list the describes its x,y,z coordinates. 

Lets calculate the net displacement of each particle from the origin (0,0,0). 

We will do this in a few different ways to compare the speeds of various methods. To probe the "speed" of a calculation we will use either the magic command `%time` or `%timeit`. 


First let's consider a simple function that uses nested for loops to perform the calculation.  

In [3]:
def disp_lists_loops(positions):
    mag = []
    for i in range(np.shape(positions)[0]):
        cur_mag = 0
        for j in range(np.shape(positions)[1]):
            cur_mag += positions[i][j]**2
        mag.append(np.sqrt(cur_mag))
    return mag

In [4]:
mag = %time disp_lists_loops(positions);

CPU times: user 4min 37s, sys: 30.9 ms, total: 4min 37s
Wall time: 4min 37s


**Wow!** that took a long time to just get the magnitude of our particle displacements! We have effectively gone through Nx3 loops for this calculation, which can be really cumbersome as the number of particles increases. 

<font color=red>**BEWARE:**</font> The overhead of this **DOESN'T** scale linearly with N!!!

Lets tweak our function a little bit to see if we can speed things up.

In [15]:
def disp_lists_loops_2(positions):
    mag = []
    for position in positions:
        cur_mag = 0
        for j in position:
            cur_mag += j**2
        mag.append(np.sqrt(cur_mag))
    return mag

In [16]:
mag_2 = %timeit disp_lists_loops_2(positions);

36.1 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This was substantially faster! All we did here was change the form of the iterator we were using in each loop. Instead of looping for a given # of iterations, we looped through the given objects in each of the lists.

From this we can see that a simple syntax change can dramatically alter the speed at which our code is executed, and we have no penalties for writing in this way. 

Why is this? The `range` function is a generator. Generators load things in one at a time and as a result are much slower. However, using generators can save RAM if you can't load the full dataset in at once.

That being said, can we make this faster?

# List comprehension

The same loop can also be written with a list comprehension. You can use list comprehension to replace many `for` and `while` blocks. List comprehension is faster because it is optimized for the Python interpreter to spot a predictable pattern during looping.

Let's test it out!

In [7]:
def disp_lists_listcomp(positions):
    mag = []
    for position in positions:
        cur_mag = sum([j**2 for j in position])
        mag.append(np.sqrt(cur_mag))
    return mag

In [8]:
mag_3 = %timeit disp_lists_listcomp(positions);

39.8 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Hmmm.... That didn't seem to help much. It is fairly comparable in speed to the loop we had before. We do have another loop that we can use list comprehension on.

In [9]:
def disp_lists_listcomp_2(positions):
    mag = [np.sqrt(sum([j**2 for j in position])) for position in positions]
    return mag

In [10]:
mag_4 = %timeit disp_lists_listcomp_2(positions);

37.2 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


We have a marginal increase in speed. This is comparable to our second looping function, but is still orders of magnitude faster than using our generator loops! In general list comprehension should be used when you're only doing one calculation in a loop. 

Be careful using nested list comprehension because it can very easily get confusing with a bunch of iterators floating around.

# Working with Vectors 

Next we will turn our attention to more complicated data. Our toy example of measuring particle displacement will still be useful (though this will seem a bit ridiculous).

When doing more complicated data analysis we often would like to be able to search and query our data. This makes packages like <font color=blue> Pandas </font> really appealing because they provide an avenue through which searchable dataframes can be generated. 

Let's turn our list of particle positions into a dataframe!

In [11]:
import pandas as pd

df_positions = pd.DataFrame(positions)
df_positions.columns = ['x','y','z']

Now we can look at the top of the dataframe to see how we might search through data.

In [12]:
df_positions.head()

,x,y,z
0,-1.010998,-0.133302,-0.987425
1,-0.112958,-2.799688,-0.570525
2,1.293490,1.876842,0.842202
3,-0.860726,0.042545,-0.563780
4,-1.219567,-2.108676,-1.636880


Notice that we have column headers labeled 'x', 'y', and 'z'. This allows us to make complicated queries of our data. Let's pull only the values of the dataset that have positive x values!

In [13]:
df_positions.loc[df_positions['x']>0]

,x,y,z
2,1.293490,1.876842,0.842202
5,0.849294,0.359905,0.651790
6,1.857685,-1.594209,-0.343060
8,1.139232,-0.603423,-0.856033
10,0.927639,-0.617204,-0.746339
11,0.201558,1.833803,-1.576334
12,2.829300,1.644372,-1.854723
13,0.465538,-0.473864,0.352223
14,0.122849,0.824778,-0.059689
15,1.113508,-0.367530,0.102336


This is a nice feature that we won't use much for something simple like just looking at displacements, but becomes more functional with more complicated analysis. Now let's see how our displacement code that we just wrote compares now that we are using a dataframe.

In [14]:
mag_5 = %timeit disp_lists_listcomp_2(df_positions);

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

Oh no! We need to tweak our function a bit to work with DataFrame types. DataFrames query through columns first. That means we have to iterate through 

In [17]:
def disp_df_listcomp(positions):
    mag = [np.sqrt(sum([position[j]**2 for j in positions.columns])) for index,position in positions.iterrows()]
    return mag

In [18]:
mag_5 = %timeit disp_df_listcomp(df_positions);

2.74 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That takes quite a bit longer than our simple list example! This is because we have traded the convenience of a searchable object for the cost of overhead. It simply takes longer to move through the object because it has a more complicated structure. 

Fortunately we can utilize <font color=blue> numpy </font> and the array datatype to speed up the calculation.

In [19]:
def disp_df_vectors(positions):
    mag = np.sqrt(np.sum(positions.values**2,axis=1))
    return mag

In [20]:
mag_7 = %timeit disp_df_vectors(df_positions);

119 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


That is the fastest we have seen yet! By using the `values` property of our DataFrame, we have a large numpy array. Because it is an array we can do vectorized calculations that neglect the need for the loops that we previously had. 

This means that we are able to have our searchable cake and eat it too! 

The benefits of vectorization can be even more dramatic if we are performing our calculation are very large datasets.

This concludes the "quick" fixes to slow code. Now we will turn our attention to the use of parallelization in our code!

## <font color=lightblue>**Pro Tip:**</font> 
You should always strive to optimize your code as much as possible before turning towards hardware acceleration. This will do more for you in the long-term and will enfore good programming habits!

# CPU Parallelization

We have seen how syntax changes have caused speed ups, but sometimes that just isn't enough. It is also useful to know how to write code that is parallelizable. I will focus on how to parallelize analysis code written in Python. 

Let's focus on the package <font color=blue> multiprocessing </font>. To effectively use multiprocessing we need to write a helper function that performs the desired calculation on each CPU node. 

<font color=red>**NOTE:**</font> Multiprocessing can replace loops, but only when one iteration of the loop does not depend on another!

Let's try using multiprocessing on our displacement example from above.

In [21]:
from multiprocessing import Pool

def disp_helper(pos):
    return np.sqrt(sum([p**2 for p in pos]))

def disp_multi(positions,n_cores=1):
    with Pool(processes=n_cores) as p:
        mag = p.map(disp_helper, positions)
    return mag

In [22]:
mag_8 = %timeit disp_multi(positions,n_cores=2)
mag_8 = %timeit disp_multi(positions,n_cores=4)
mag_8 = %timeit disp_multi(positions,n_cores=6)
mag_8 = %timeit disp_multi(positions,n_cores=8)

108 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
113 ms ± 605 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
118 ms ± 831 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
122 ms ± 591 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


What the heck is going on!? Our code is slowing down as we increase the number of cores we are using! 

If we dig deeper and look at how much each core is utilized you will see something interesting. In doing this my computer cores were only under ~10% load. Because the load is so small we actually waste more time in coordinating threads than we would spend doing the calculation. 

This is a good cautionary tale so we don't spend time multithreading everything that we write. Sometimes simpler is better!

This of course will change when the calculations being performed in the loop are large or complicated and require more of the CPU resource. We really see the benefit of multithreading when a single CPU thread is working under full load.

# GPU programming in Python

We just learned how to use our CPU to it's fullest potential by utilizing as many threads as possible to perform calculations (and when not to use it's full potential). Sometimes this can still be slow though. The reason being is that most modern computers only have 4-8 cores which translates into 8-16 threads. While this can provide substantial speed increases, we are often dealing with tens or hundreds of thousands of particles. If time-stepping is important then this number gets multiplied by the number of timesteps, which can easily result in another factor of 1000. When the scales are this big a x8-16 speed increase is definitely noticeable, but still isn't anything to write home about. 

That is where the GPU comes in! GPUs, unlike CPUs, have thousands of threads that can be used for multiprocessing. In this tutorial we will focus on Nvidia GPUs as these are the most common. Nvidia has a proprietary language known as [CUDA](https://docs.nvidia.com/cuda/), which allows you to interface with the GPU architecture. Since there are so many threads working in parallel, there is a more advanced hierarchy needed for getting them to coordinate and work together. 

Here we will focus on using CUDA in Python through the [numba](https://numba.pydata.org/) package. There is some overhead associated with this package as it is a wrapper, but it is much easier than writing CUDA code from scratch. If you wish to write direct CUDA code in C++, then please refer to the documentation above. For documentation related to installing numba for use with CUDA please refer to the page [here](http://numba.pydata.org/numba-doc/latest/cuda/index.html). 

<font color=blue>Numba</font> works like most Python packages, allows you to vectorize your code for faster CPU implementation or for GPU implementation through the use of decorators used in front of functions. For CUDA implementation we will use the `cuda.jit` decorator. 

In [23]:
from numba import cuda
import math

def disp_GPU(positions):
    # Make sure our DF is a Numpy array
    positions = positions.values
    
    # Initialize some CUDA kernel parameters
    blocks_per_grid = 30
    threads_per_block = 128
    
    # Initialize our output array
    mag = np.zeros(len(positions))
    
    # Run GPU kernel
    disp_GPU_calc[blocks_per_grid, threads_per_block](positions,mag)
    
    return mag

@cuda.jit
def disp_GPU_calc(positions, mag):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(start,len(positions),stride):
        mag[i] = math.sqrt(positions[i][0]**2 + positions[i][1]**2 + positions[i][2]**2)
    

In [24]:
mag_9 = %timeit disp_GPU(df_positions);

1.13 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


That was pretty fast! But why was it slower than just our regular vectorized code? While the GPU is fast, there is a limiting step of communication between the CPU and GPU. That means that the more we can do on the GPU or 'device', the more dramatic our speed increase will be. If you write some GPU code and it doesn't quite seem fast enough, then see if you're calling anything that requires host-device communication. 

For our simple example we aren't quite doing any calculation that is complex enough to really benefit from the highly parallelized nature of the GPU. It doesn't take much complexity for this to win out over other methods though.

Similarly to using loops, our GPU code can benefit from some simple syntax changes. Let's first look at the type of data we are using.

In [25]:
df_positions.values.dtype

dtype('float64')

Our numpy array has saved our values as 'float64'. These are fairly large values to store and can give the GPU some trouble from a speed perspective. Also, most of the GPU functions or values will be written as 'float32', so we have already introduced 32-bit error. Let's change our numpy arrays all to float32 values.

In [26]:
def disp_GPU_32bit(positions):
    # Make sure our DF is a Numpy array
    positions = positions.values.astype(np.float32)
    
    # Initialize some CUDA kernel parameters
    blocks_per_grid = 30
    threads_per_block = 128
    
    # Initialize our output array
    mag = np.zeros(len(positions), dtype=np.float32)
    
    # Run GPU kernel
    disp_GPU_calc[blocks_per_grid, threads_per_block](positions,mag)
    
    return mag

In [27]:
mag_10 = %timeit disp_GPU_32bit(df_positions);

758 µs ± 22.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


From this we see that our code got faster by roughly 20%. We can do one more thing to speed the code up. We can actually turn our data to CUDA arrays before feeding it to the GPU. This allows the GPU interpreter to work more efficiently. 

In [28]:
def disp_GPU_cuda_vec(positions):
    # Make sure our DF is a Numpy array
    cud_positions = cuda.to_device(positions.values.astype(np.float32))
    
    # Initialize some CUDA kernel parameters
    blocks_per_grid = 30
    threads_per_block = 128
    
    # Initialize our output array
    mag = cuda.to_device(np.zeros(positions.values.shape[0], dtype=np.float32))
    
    # Run GPU kernel
    disp_GPU_calc[blocks_per_grid, threads_per_block](cud_positions,mag)
    
    mag = mag.copy_to_host()
    
    return mag

In [29]:
mag_11 = %timeit disp_GPU_cuda_vec(df_positions);

632 µs ± 5.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We got another ~20% speed increase! Now our GPU code is comparable to the vectorized code that we have, even for such a simple example. 

These kinds of speed boosts can be dramatic (see [HOOMD](https://hoomd-blue.readthedocs.io/en/stable/index.html) for a clear example of how GPUs can really change the game when it comes to computation).

If you want more references and resources related to <font color=blue>numba</font> please checkout the [GTC 2017](https://github.com/ContinuumIO/gtc2017-numba) github for jupyter notebooks that you can work through.

Written by: adulaney